# FT-LLM

FT-LLM은 축구 지식에 강점이 있는 챗봇을 위해 개발된 LLM입니다.

다양한 LLM을 backbone으로 사용하며, retrieval을 통해 사용자의 질문에 적합한 답변을 생성하는 강점을 가집니다.

코드 상으론 FT-LLM은 다음과 같은 단계로 작동합니다.

1. 사용자의 query 입력
2. LLM이 조금 더 문장을 잘 이해할 수 있도록 사용자 query를 rephrase
3. Rephrase된 Query와 과장 연관성이 높은 축구 관련 document를 DB (json 파일)로 부터 retrieve
4. 가장 연관성이 높다고 판단이 되는 document K개와 rephrased된 query를 함께 다시 LLM에 입력
5. 최종적으로 document의 내용을 기반으로 생성된 response의 내용이 사실인지 Fact Checking (단 현 단계에선 fact checking은 생략함)

run.py의 main 함수 참고할 것

In [1]:
%cd /Project/

/Project


In [2]:
from run import main
from model.model import load_models

In [3]:
import yaml

cfg = yaml.load(open('./config/runner_chatgpt.yaml','r'), Loader=yaml.FullLoader)

In [5]:
from glob import glob
import json
import numpy as np

def load_json_data(data_dir):
    """
    Load multiple JSON files from the folder and merge.
    """

    files = glob(data_dir+"/*/*.json")
    files.sort()
    all_data = []
    for file_path in files:
        #print("Loading: ",file)
        #file_path = os.path.join(data_dir, file)
        with open(file_path, "r", encoding = "utf-8-sig") as f:
            doc = json.load(f)
        all_data.append(doc)
        #all_data += doc
    return all_data

In [6]:
doc = load_json_data('./json_data')

In [44]:
doc_content = [item['title'] + item['contents'] for item in doc]


In [50]:
doc_content

['\'레알 이적설\' 음바페, 17일 휴가 마치고 PSG 복귀 예정...이강인과 호흡 맞출까\n사진=게티 이미지(좌), PSG(우)킬리안 음바페가 파리 생제르맹(PSG) 프리시즌 훈련에 합류할 예정이다.PSG 소식을 전하는 \'PSG Talk\'는 16일(한국시간) "음바페는 17일에 휴가를 마치고 돌아올 것이며 PSG에 다시 합류할 것으로 예상된다. 하지만 그의 미래는 여전히 불확실하다. PSG의 입장은 확고하지만, 음바페의 계약 연장에 대한 클럽의 입장은 변하지 않았다"고 보도했다.올여름 음바페의 거취를 놓고 전 세계가 집중하고 있다. 음바페의 이적 사가가 처음 시작된 건 지난달 13일부터였다. 유럽 축구 소식에 능통한 파브리시오 로마노 기자는 "음바페는 PSG에 그의 결정을 알렸다. 그는 2025년까지 1년 연장 옵션을 발동하지 않을 것이다. 그는 2024년에 계약이 만료된다"고 전했다.이어 "PSG의 입장은 분명하다. 음바페가 이번 여름 재계약을 맺거나 매각시키는 것이다. PSG는 2024년에 음바페를 자유계약(FA) 상태로 잃을 의사가 전혀 없다. PSG는 음바페가 결정에 어떠한 대화도 요구하지 않았다는 것에 놀랐다"고 덧붙였다.영국 공영방송 \'BBC\'도 "음바페는 PSG에 재계약을 하지 않겠다고 말했다. 음바페가 재계약 여부를 알리는 기한은 7월 31일까지다. 하지만 몇 달 동안 진행된 대화 끝에 연장을 거절하는 서한을 보냈다"고 전했다.그러나 음바페는 해당 보도들을 전면 부인했다. 그는 지난달 13일 개인 SNS를 통해 "거짓말이다. 나는 이미 PSG에서 행복하다. 다음 시즌에도 계속할 것이라고 말했다"며 반박했다. 공식 인터뷰를 통해 다시 한번 어필했다.지난달 17일 열린 지브롤터와 맞대결을 앞두고 음바페는 "나는 이미 대답했다. 내 목표는 잔류하는 것이다. 지금은 그것이 나의 유일한 선택이다. 나는 내가 보낸 편지가 누군가를 죽인다고 생각하지 않는다. 그것에 대해 신경 쓰지 않는다. 누군가를 화나게 할 것이라고 생각하지 않았다"며 강조했다

In [22]:
corpus = []
doc_id_ref = []

for idx, passage in enumerate(doc):
    
    tokenized_doc = passage['contents'].split('.')
    corpus.append(tokenized_doc)
    doc_id_ref.append([idx]*len(tokenized_doc))

In [23]:
# unlist doc
corpus = [item for sublist in corpus for item in sublist]

# unlist doc_id_ref
doc_id_ref = [item for sublist in doc_id_ref for item in sublist]
doc_id_ref = np.array(doc_id_ref)

In [24]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(corpus)

In [25]:
query = "리오넬 메시의 통산 골 개수는?"
scores = bm25.get_scores(query)

In [26]:
top_k_list = np.argpartition(scores,-5)[-5:]
top_k_indices = doc_id_ref[top_k_list]

In [40]:
raw_reference_list = []

for idx in top_k_indices:
    raw_reference = doc[idx]
    raw_reference_list.append(raw_reference)

In [ ]:
raw_reference

In [28]:
corpus[top_k_list]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 corpus[top_k_list]                                                                           │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: only integer scalar arrays can be converted to a scalar index

In [27]:
top_k_list

array([5192352, 2103500, 5194612,  921904, 5189760])

In [8]:
history = []
history_rewrite_request = []
history_url = []
solver = load_models(model_config_path = cfg['Model_config']['llm_configs'],#"./config/llm_chatgpt.json", 
                    device = cfg['Model_config']['device']#"cuda:1"
                    )

In [9]:
while True:
    query = input('Enter any inquries! (quit to exit):')

    if query == 'quit':
        print('Chat Quitted')
        break

    src = query, history, history_rewrite_request, history_url
    
    history, history_rewrite_request, history_url = main(src, solver, cfg)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


최초 입력:  첼시의 챔피언스 리그 우승 횟수는 몇 번이야?
챗봇에 적합하게 재구성된 입력:첼시의 챔피언스 리그 우승 횟수는 몇 번이야?
Checking relevant Sources
Extracting relevant Sources
챗봇:  첼시의 챔피언스 리그 우승 횟수는 질문에 대한 참고 자료에 언급되어 있지 않습니다.
최초 입력:  


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


챗봇에 적합하게 재구성된 입력:첼시의 챔피언스 리그 우승 횟수는 몇 번이야?
Checking relevant Sources


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 │                                                                                           │
│    8 │   src = query, history, history_rewrite_request, history_url                              │
│    9 │                                                                                           │
│ ❱ 10 │   history, history_rewrite_request, history_url = main(src, solver, cfg)                  │
│   11                                                                                             │
│                                                                                                  │
│ /Project/run.py:20 in main                                                                       │
│                                                                                                  │
│    17 │   history_rewrite_request.append(new_query)                                              │
│    18 │                                                                                          │
│    19 │   print('Checking relevant Sources')                                                     │
│ ❱  20 │   raw_lists = solver.Retrieve_documents(                                                 │
│    21 │   │   │   │   │   │   │   │   │   query = new_query,                                     │
│    22 │   │   │   │   │   │   │   │   │   data_dir = config['Retriever_config']['data_dir'], #   │
│    23 │   │   │   │   │   │   │   │   │   top_k = config['Retriever_config']['top_k'],           │
│                                                                                                  │
│ /Project/model/model.py:124 in Retrieve_documents                                                │
│                                                                                                  │
│   121 │   │   model.load(check_point_dir)                                                        │
│   122 │   │   model.eval()                                                                       │
│   123 │   │                                                                                      │
│ ❱ 124 │   │   builder = Index_Builder(model = model,                                             │
│   125 │   │   │   │   │   │   │   │   tokenizer=tokenizer,                                       │
│   126 │   │   │   │   │   │   │   │   data_dir = data_dir,                                       │
│   127 │   │   │   │   │   │   │   │   index_save_dir = save_dir,                                 │
│                                                                                                  │
│ /Project/model/retriever.py:59 in __init__                                                       │
│                                                                                                  │
│    56 │   │   self.index_save_dir = index_save_dir                                               │
│    57 │   │   self.use_content_type = use_content_type                                           │
│    58 │   │   os.makedirs(index_save_dir, exist_ok = True)                                       │
│ ❱  59 │   │   self.all_docs = load_json_data(self.data_dir)                                      │
│    60 │   │   self.doc_loader = self.load_data(batch_size)                                       │
│    61 │                                                                                          │
│    62 │   def load_data(self, batch_size):                                                       │
│                                                                                                  │
│ /Project/model/retriever.py:185 in load_json_data          